<p>
<img src="https://drive.google.com/uc?id=1ZHUKcwzA5c-PKUwoGmBG3TsHk6TmZ-Y0" align="left" width="250">
</p>
<p>
<h1><b><center>Mecánica Celeste</center></b></h1>
<h2><b><center>Prof. Jorge I. Zuluaga</center></b></h1>
<h2><b><center>Tarea 3</center></b><h2>
<h3><b><center>La aproximación de Apophis</center></b><h3>
<h5><center><b>Asignada</b>: 19 de Septiembre de 2023</center><h5>
<h5><center><b>Entrega</b>: 3 de Octubre de 2023</center><h5>
</p>

<hr/>
<b>Nombre</b>:
<br/>
<b>Cédula</b>:
<br/>
<b>Última actualización</b>:
<hr/>

## Enunciado

**Objetivo**: El objetivo de esta tarea es utilizar la solución analítica del problema de los dos cuerpos para estudiar la aproximación del asteroide peligroso **99942 Apophis (2004 MN4)** durante el 13 de abril del año 2029.

**Procedimiento**:

1. Obtenga el vector de estado del asteroide Apophis el día de su cumpleaños en 2023 a las 00:00:00 UTC con respecto al Sol, así como el vector de estado de la Tierra respecto al mismo sistema de referencia.

2. Determine los elementos orbitales del asteroide y los elementos órbitales osculatrices de la Tierra usando el procedimiento aprendido en clase. Estos elementos orbitales serán los que usará a partir de ahora en todos los cálculos que use la teoría del problema de los dos cuerpos. Es importante que use el procedimiento en clase en lugar de obtener simplemente los elementos de astroquery.

3. Usando los elementos orbitales de ambos cuerpos determine la mínima distancia a la que se encontarán ambas órbitas en cualquier momento del futuro. A esta distancia se la llama MOID, por el acrónimo en inglés *Minimum orbital intersection distance*.

4. Para saber cuándo se producirá la máxima aproximación entre los dos cuerpos es necesario sin embargo resolver el problema de los dos cuerpos en el tiempo. Para ello use la ecuación de Kepler y los métodos aprendidos en clase para propagar hacia el futuro la posición de la Tierra y del asteroide en el tiempo y determinar la fecha, hora y distancia de la próxima aproximación. Para ello avance primero de mes en mes por ejemplo, y luego, cerca a la aproximación máxima, de día en día, de hora en hora, hasta encontrar el punto de mínima distancia. ¿En qué fecha, hora y distancia predice usted que se producirá la máxima aproximación?

5. Una vez encontrada la fecha y hora de la aproximación obtenida con el problema exclusivamente con la solución analítica del problema de los dos cuerpos (la que estamos viendo aquí), compárela con la que se obtiene recuperando de astroquery las posiciones de la Tierra y Apophis desde su cumpleaños hacia el futuro. Es decir, repita el procedimiento anterior, pero en lugar de obtener las posiciones a partir de la teoría del problema de los dos cuerpos, recupere las posiciones usando astroquery.  ¿Qué tan diferentes son los resultados de este procedimiento y del anterior? ¿podría explicar las discrepancias?

**Resultados esperados**:

- Este notebook con el procedimiento debidamente comentado.

- Datos mínimos:
  - MOID de la órbita de Apophis y de la Tierra calculado con la teoría del problema de los 2 cuerpos.
  - Fecha, hora y distancia mínima de aproximación calculado con la teoría del problema de los 2 cuerpos.
  - Fecha, hora y distancia mínima de aproximación calculado con los datos obtenidos de Astroqery.

- Gráficos mínimos:
  - Distancia entre Apophis y la Tierra como función del tiempo desde la fecha de nacimiento hasta la máxima aproximación, calculada con la teoría del problema de los dos cuerpos y con los datos de Astroquery.

- Gráficos opcionales:
  - Animaciones de la aproximación.

- En caso de utilizar ayudas como `ChatGPT` o códigos de `Stack Over Flow` debe citar debidamente la fuente, pero también entender a cabalidad el código, el profesor puede pedir justificación.

instalemos las librerias necesarias:

In [119]:
# python3 -m pip install -q pymcel astroquery astropy plotly


Importación de librerias:

In [120]:
import pymcel as pc
import numpy as np
import pandas as pd
import spiceypy as spy
import matplotlib.pyplot as plt
from astroquery.jplhorizons import Horizons
from astropy.time import Time
from astropy import constants as c,units as u

Constantes:

In [121]:
# Constante para convertir m^3 a km^3
m3_km3 = 10**9

# Gravitational constant en m^3/(kg s^2)
G = c.G.value

# Convertir GM a masa en Kg
GM2M = 1 / (G / m3_km3)

# Encontrar la masa del sol con alta precisión
# Descargar la librería de SPICE que contiene datos necesarios
!wget -q https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/gm_de440.tpc -O gm_de440.tpc
spy.furnsh('gm_de440.tpc')

# Obtener la información de la masa del sol desde la base de datos de SPICE
n, dato = spy.bodvrd('sun', 'GM', 1)

# Convertir unidades
M_sun = dato[0] * GM2M

In [122]:
# Para pasar de grados a radianes y viceversa
deg2rad = np.pi/180
rad2deg = 1/deg2rad
m3_km3= 10**9 #m^3 a km^3

# factores de conversión
day = 86400 # segundos
au = c.au.value # metros
G = c.G.value # m^3/(kg s^2)

Unidades Canónicas:

In [123]:
UL = au # m
UM = M_sun # kg
UT = (UL**3/(G*UM))**0.5 # segundos

# Unidades derivadas
UV = UL/UT

# Entonces el mu en unidades canónicas es
mu = 1

## Solución

### 1. Obtenga el vector de estado del asteroide Apophis el día de su cumpleaños en 2023 a las 00:00:00 UTC con respecto al Sol, así como el vector de estado de la Tierra respecto al mismo sistema de referencia.

In [124]:
Horizons(
    id='2004 MN4', location='@sun', epochs=jd
).vectors()

targetname,datetime_jd,datetime_str,H,G,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,mag,---,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str24,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
99942 Apophis (2004 MN4),2460151.5,A.D. 2023-Jul-26 00:00:00.0000,19.09,0.24,0.1576713962490502,0.936654275657962,-0.04620787634986293,-0.01626952578813921,0.006037769768090685,-0.0007074045257305857,0.005492261852344384,0.9509556609749115,0.003283803662357464


In [125]:
# Mi cumpleaños en 2023 fue el 2023-07-26
jd = Time('2023-07-26 00:00:00').jd
# Día Juliano de mi cumpleaños en 2023

# Consulta el vector de estado de Aphophis en este día
X0_aphophis_df = Horizons(
    id='2004 MN4', location='@sun', epochs=jd
).vectors().to_pandas()

# Construyendo nuestros vectores fila con el estado inicial de Aphophis
X0_aphophis = np.array(X0_aphophis_df[['x','y','z']]*au/UL)[0]  # Para la posición
X0_aphophis_norm = np.linalg.norm(X0_aphophis)  # La norma

V0_aphophis = np.array(X0_aphophis_df[['vx','vy','vz']]*(au/day)/UV)[0]  # Para la velocidad
V0_aphophis_norm = np.linalg.norm(V0_aphophis)  # La norma

# Consulta el vector de estado de la tierra en este día
X0_earth_df = Horizons(
    id='399', location='@sun', epochs=jd
).vectors().to_pandas()

# Construyendo nuestros vectores fila con el estado inicial de Aphophis
X0_earth = np.array(X0_earth_df[['x','y','z']]*au/UL)[0]  # Para la posición
X0_earth_norm = np.linalg.norm(X0_earth)  # La norma

V0_earth = np.array(X0_earth_df[['vx','vy','vz']]*(au/day)/UV)[0]  # Para la velocidad
V0_earth_norm = np.linalg.norm(V0_earth)  # La norma

### 2. Determine los elementos orbitales del asteroide y los elementos órbitales osculatrices de la Tierra usando el procedimiento aprendido en clase. Estos elementos orbitales serán los que usará a partir de ahora en todos los cálculos que use la teoría del problema de los dos cuerpos. Es importante que use el procedimiento en clase en lugar de obtener simplemente los elementos de astroquery.

### Encontremos los elementos orbitales de apophis


#### Primero calculemos los vectores clave del sistema

In [126]:
# Calculamos el M.A.R.E
hvec_aphophis = np.cross(X0_aphophis, V0_aphophis)
h_aphophis = np.linalg.norm(hvec_aphophis)

# Calculando el vector nodal
nvec_aphophis = np.cross([0, 0, 1], hvec_aphophis)  # [0, 0, 1] es el vector x unitario
n_aphophis = np.linalg.norm(nvec_aphophis)
h_aphophis

0.016218441820317883

A partir de estos vectores podemos calcular los elementos orbitales para apophis

In [127]:
# Calculando el vector de Laplace
evec_aphophis = (np.cross(V0_aphophis, hvec_aphophis) / mu -
                 X0_aphophis / X0_aphophis_norm)
e_aphophis = np.linalg.norm(evec_aphophis)  # Excentricidad

# Inclinación
I_aphophis = np.arccos(hvec_aphophis @ [0, 0, 1] / h_aphophis) * rad2deg # [0, 0, 1] es el vector z unitario

# Longitud del nodo ascendente
Omegap_aphophis = np.arccos(nvec_aphophis @ [1, 0, 0] / n_aphophis) # [1, 0, 0] es el vector x unitario
Omega_aphophis = (Omegap_aphophis if nvec_aphophis[1] >= 0 
                  else 2 * np.pi - Omegap_aphophis)
Omega_aphophis *= rad2deg

# Argumento del periapsis
omegap_aphophis = np.arccos(
    evec_aphophis @ nvec_aphophis / (e_aphophis * n_aphophis))
omega_aphophis = (omegap_aphophis if evec_aphophis[2] >= 0 
                  else 2 * np.pi - omegap_aphophis)
omega_aphophis *= rad2deg

# Anomalía verdadera
fp_aphophis = np.arccos(
    evec_aphophis @ X0_aphophis / (e_aphophis * X0_aphophis_norm))
f_aphophis = (fp_aphophis if X0_aphophis @ V0_aphophis > 0 
              else 2 * np.pi - fp_aphophis)
f_aphophis *= rad2deg

# Semilatus rectum
p_aphophis = h_aphophis**2 / mu


Creemos un dataframe con los elementos orbitales para Apophis

In [128]:
# Creando el DataFrame
data = {
    "Elementos orbitales Aphophis": [
        e_aphophis, I_aphophis, Omega_aphophis,
        omega_aphophis, f_aphophis, p_aphophis,n_aphophis
    ]
}
indices = ["e", "I", "Omega", "omega", "f", "p",'n']
df = pd.DataFrame(data, index=indices)

df

,Elementos orbitales Aphophis
e,0.999723
I,3.339301
Omega,203.957180
omega,56.535395
f,179.996948
p,0.000263
n,0.000945


### Encontremos los elementos orbitales de la tierra

#### Primero calculemos los vectores clave del sistema

In [129]:
# Calculamos el M.A.R.E
hvec_earth = np.cross(X0_earth, V0_earth)
h_earth = np.linalg.norm(hvec_earth)

# Calculando el vector nodal
nvec_earth = np.cross([0, 0, 1], hvec_earth)  # [0, 0, 1] es el vector x unitario
n_earth = np.linalg.norm(nvec_earth)


A partir de estos vectores podemos calcular los elementos orbitales para apophis

In [130]:
# Calculando el vector de Laplace
evec_earth = (np.cross(V0_earth, hvec_earth) / mu -
                 X0_earth / X0_earth_norm)
e_earth = np.linalg.norm(evec_earth)  # Excentricidad

# Inclinación
I_earth = np.arccos(hvec_earth @ [0, 0, 1] / h_earth) * rad2deg # [0, 0, 1] es el vector z unitario

# Longitud del nodo ascendente
Omegap_earth = np.arccos(nvec_earth @ [1, 0, 0] / n_earth) # [1, 0, 0] es el vector x unitario
Omega_earth = (Omegap_earth if nvec_earth[1] >= 0 
                  else 2 * np.pi - Omegap_earth)
Omega_earth *= rad2deg

# Argumento del periapsis
omegap_earth = np.arccos(
    evec_earth @ nvec_earth / (e_earth * n_earth))
omega_earth = (omegap_earth if evec_earth[2] >= 0 
                  else 2 * np.pi - omegap_earth)
omega_earth *= rad2deg

# Anomalía verdadera
fp_earth = np.arccos(
    evec_earth @ X0_earth / (e_earth * X0_earth_norm))
f_earth = (fp_earth if X0_earth @ V0_earth > 0 
              else 2 * np.pi - fp_earth)
f_earth *= rad2deg

# Semilatus rectum
p_earth = h_earth**2 / mu


Creemos un dataframe con los elementos orbitales para la tierra

In [131]:
# Creando el DataFrame
data = {
    "Elementos orbitales earth": [
        e_earth, I_earth, Omega_earth,
        omega_earth, f_earth, p_earth
    ]
}
indices = ["e", "I", "Omega", "omega", "f", "p"]
df = pd.DataFrame(data, index=indices)

df

,Elementos orbitales earth
e,0.016901
I,0.002396
Omega,217.128706
omega,244.908643
f,200.432183
p,0.999646


### 3. Usando los elementos orbitales de ambos cuerpos determine la mínima distancia a la que se encontarán ambas órbitas en cualquier momento del futuro. A esta distancia se la llama MOID, por el acrónimo en inglés *Minimum orbital intersection distance*.

### 4. Para saber cuándo se producirá la máxima aproximación entre los dos cuerpos es necesario sin embargo resolver el problema de los dos cuerpos en el tiempo. Para ello use la ecuación de Kepler y los métodos aprendidos en clase para propagar hacia el futuro la posición de la Tierra y del asteroide en el tiempo y determinar la fecha, hora y distancia de la próxima aproximación. Para ello avance primero de mes en mes por ejemplo, y luego, cerca a la aproximación máxima, de día en día, de hora en hora, hasta encontrar el punto de mínima distancia. ¿En qué fecha, hora y distancia predice usted que se producirá la máxima aproximación?

#### Para realizar este proceso, primero voy a realizar una o varias funciones que me calcule el vector de estado final, dados los elementos orbitales el tiempo inicial y final

##### Funciones auxiliares

In [132]:
def prediccion(jd,jdf,a,e,I,Omega,w,f,n):
    # E inicial
    E = 2*np.arctan(np.sqrt((1-e)/(1+e))*np.tan(f*deg2rad/2))
    #E=E*rad2deg

    # M = E - e sin E
    M = E - e*np.sin(E)
    M=M*rad2deg

    #Ahora para las condiciones finales:
    Mf = M + n*(jdf-jd)
    Ef,error,niter = pc.kepler_newton(Mf*deg2rad,e,delta=1e-14)
    #Ef=Ef*rad2deg

    #Encontremos la anomalía verdadera
    ff = 2*np.arctan(np.sqrt((1+e)/(1-e))*np.tan(Ef/2))
    ff=ff*rad2deg
    
    return ff
prediccion(jd,jd+365.25,1,
        e_aphophis, I_aphophis, Omega_aphophis,
        omega_aphophis, f_aphophis, 1.11198)

-179.72420691168625

In [133]:
e_aphophis, I_aphophis, Omega_aphophis,omega_aphophis, f_aphophis, n_aphophis

(0.9997233977178548,
 3.3393005993205813,
 203.957179610651,
 56.535394692801255,
 179.99694768673396,
 0.0009447047873823015)

In [134]:
queryf = Horizons(id='2004 MN4',location='@sun',epochs=jd)
elementf = queryf.elements()
elementf

targetname,datetime_jd,datetime_str,H,G,e,q,incl,Omega,w,Tp_jd,n,M,nu,a,Q,P
---,d,---,mag,---,---,AU,deg,deg,deg,d,deg / d,deg,deg,AU,AU,d
str24,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
99942 Apophis (2004 MN4),2460151.5,A.D. 2023-Jul-26 00:00:00.0000,19.09,0.24,0.1914427971442487,0.7460744636697038,3.339300599320679,203.957179610651,126.6044627878949,2460072.030268009,1.111979897875463,88.36874446324205,109.9278795917671,0.9227231679275577,1.099371872185412,323.7468597119534


In [135]:
180-147

33